In [1]:
from src.data.dataset_functions import get_dataframe_onlymods_full

modframe = get_dataframe_onlymods_full(True)
#modframe.show(5)

providing ../data/raw/2022_place_canvas_history-000000000003.csv ...
not found. need to download ../data/raw/2022_place_canvas_history-000000000003.csv.gzip ...
downloading from https://placedata.reddit.com/data/canvas-history/2022_place_canvas_history-000000000003.csv.gzip to ../data/raw/2022_place_canvas_history-000000000003.csv.gzip
unpacking ../data/raw/2022_place_canvas_history-000000000003.csv.gzip into ../data/raw/2022_place_canvas_history-000000000003.csv
deleting ../data/raw/2022_place_canvas_history-000000000003.csv.gzip
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4b8e5ec7-07a4-4d13-9c76-c08c366c60bd;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 72ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

+--------------------+---+----+---+----+------+-----------+
|             user_id| x1|  y1| x2|  y2|     t|pixel_color|
+--------------------+---+----+---+----+------+-----------+
|m8NEcPbf5XRV5ppeu...|298|1805|329|1839|216904|    #FFB470|
|LKS2u3QL2N3Olv7rn...|257|1736|296|1780|217130|    #FFB470|
|q/Dk6lmcXm8bcDbNI...|298|1770|334|1803|216809|    #FFB470|
|HkR0yRQUJ1wsjh4Zo...|251|1805|296|1812|217236|    #FFF8B8|
|7JiQyrONpFJphvBEP...|271|1835|296|1859|217371|    #FFF8B8|
+--------------------+---+----+---+----+------+-----------+



In [2]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)

providing ../data/raw/2022_place_canvas_history-000000000003.csv ...
../data/raw/2022_place_canvas_history-000000000003.csv is already in data/raw


In [3]:
from src.features.feature_functions import get_latestpixels_from_box
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import functools
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Empty_Dataframe').getOrCreate()

#Mergen zweier DataFrames
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

#Ja, datacollect ist ineffizient ABER es gibt nicht so viele Zensuren, daher sollte das halb so wild sein!
mods = modframe.drop('pixel_color').drop('user_id').collect()

emp_RDD = spark.sparkContext.emptyRDD()

# Create an expected schema
columns = StructType([StructField('x',
                                IntegerType(), True),
                    StructField('y',
                                IntegerType(), True),
                    StructField('user_id',
                                StringType(), True),
                    StructField('t',
                                LongType(), True),
                    StructField('pixel_color',
                                StringType(), True)])


censoredData = spark.createDataFrame(data = emp_RDD,
                                    schema = columns)

#DataFrame für alle gemachten Zensuren mit den betroffenen Pixel generieren
for row in mods:
    min_x,min_y,max_x,max_y,tz = list(row)
    censoredData = unionAll([censoredData,get_latestpixels_from_box(dataFrame,min_x,min_y,max_x,max_y,tz)])

In [4]:
#censoredData.show(5)
total_censors = censoredData.count() #Anzahl der betroffenen Pixel speichern (später für Statistik wichtig!)

+---+----+--------------------+------+-----------+
|  x|   y|             user_id|     t|pixel_color|
+---+----+--------------------+------+-----------+
|298|1805|8Ncd5vj/7mJHwq1z6...|216819|    #FFB470|
|298|1809|Wf5SrgaWTWXW4Yn5Q...|216828|    #FF99AA|
|298|1811|ZYSDFfRaU6mr2/GHW...|216682|    #FFB470|
|298|1812|dd7OsqTULsU0SFGsv...|216853|    #FFB470|
|298|1834|93RvWf9B2YUC7tubU...|216691|    #000000|
+---+----+--------------------+------+-----------+
only showing top 5 rows



<div style="color:#4e31ce">

So bis hier wurde die Funktion nun für den GESAMTEN Datensatz verallgemeinert....

Jetzt muss DataFrame mit user_ids für Bots generiert werden und dann anhand eines Joins geschaut werden, welche Pixel zu Bots gehören.

Dann kann count() für das DataFrame mit den Bots angewendet werden und count() für die Ausgangstabelle mit den von der Zensur betroffenen
Pixeln, sodass das eine durch das andere geteilt werden kann um entsprechend das Verhältnis rauszubekommen!
</div>

<div style="color:#4e31ce">
Dann beginnen wir mal das DataFrame mit den user_ids für die Bots zu generieren.
</div>

<div>
botDiff = timeDiffReduced.alias('tdr1').join(timeDiffReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'),'inner').join(timeDiffReduced.alias('tdr3'),F.col('tdr1.user_id') == F.col('tdr3.user_id'), 'inner')


botDiff = botDiff.where('tdr1.diff == tdr2.diff').where('tdr1.t2 == tdr2.t1').where('tdr3.diff == tdr1.diff').where('tdr2.t2 == tdr3.t1')
</div>

In [5]:
#DataFrame ist schon vorhanden, siehe oben!
timeDiffFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
timeDiffFrame = timeDiffFrame.where('df2.t > df1.t')
timeDiffFrame = timeDiffFrame.orderBy(['df1.user_id', 'df1.t'])
timeDiffFrame = timeDiffFrame.select(F.col('df1.user_id'),F.col('df1.t').alias('t1'),F.col('df2.t').alias('t2'),(F.col('df2.t') - F.col('df1.t')).alias('diff'))
#timeDiffFrame.printSchema()

In [6]:
timeDiffReduced = timeDiffFrame.dropDuplicates(['user_id', 't1'])
timeDiffReduced = timeDiffReduced.orderBy(['user_id', 't1'])

In [7]:
botDiff = timeDiffReduced.alias('tdr1').join(timeDiffReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'),'inner').join(timeDiffReduced.alias('tdr3'),F.col('tdr1.user_id') == F.col('tdr3.user_id'), 'inner')
botDiff = botDiff.where('tdr1.diff == tdr2.diff').where('tdr1.t2 == tdr2.t1').where('tdr3.diff == tdr1.diff').where('tdr2.t2 == tdr3.t1')
botDiff = botDiff.select('tdr1.user_id','tdr1.diff')
botDiff = botDiff.dropDuplicates(['user_id'])
botDiff = botDiff.orderBy(['user_id'])
#botDiff.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- diff: long (nullable = true)



In [8]:
botDiff = botDiff.drop("diff")
#botDiff.printSchema()

root
 |-- user_id: string (nullable = true)



<div style="color:#4e31ce">
Das DataFrame botDiff ist dann unsere Liste an Bots.
</div>

<div style="color:#4e31ce">
Jetzt müssen die beiden DataFrames censoredData und botDiff gejoined werden übder die user_ids, sodass nur noch Bots erhalten bleiben.
</div>

In [13]:
joinedCensors = censoredData.alias('cd').join(botDiff.alias('bd'),F.col('cd.user_id') == F.col('bd.user_id'),'inner')
#joinedCensors.printSchema()

root
 |-- x: long (nullable = true)
 |-- y: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)
 |-- user_id: string (nullable = true)



In [14]:
joinedCensors = joinedCensors.drop('bd.user_id')
#joinedCensors.show(5)

22/07/08 11:11:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/08 11:11:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/08 11:11:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/08 11:11:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---+---+-------+---+-----------+-------+
|  x|  y|user_id|  t|pixel_color|user_id|
+---+---+-------+---+-----------+-------+
+---+---+-------+---+-----------+-------+



In [16]:
#Jetzt nur noch Menge zählen für joinedCensors
bots = joinedCensors.count()
#falls Bots zensiert wurden, muss bots != 0 sein
if(bots is not None or bots == 0):
    censoredBots = bots / total_censors
    censoredUsers = 100 - censoredBots
#wurden keine Bots zensiert (sehr unwahrscheinlich) ist bots = 0
else: 
    censoredBots = 0
    censoredUsers = 100
print("Von den zensierten Pixeln wurden ",censoredBots,"% von Bots und ",censoredUsers,"% von Nutzern gesetzte Pixel zensiert.")

22/07/08 11:18:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/08 11:18:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/08 11:18:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/08 11:18:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Von den zensierten Pixeln wurden  0.0 % von Bots und  100.0 % von Nutzern gesetzte Pixel zensiert.
